## Board representation
- https://codereview.stackexchange.com/a/87488
    - I decided to use map to store tic-tac-toe board because this representation seemed to me to be more versatile.
    - It's also possible to use vectors to store board states, which I think is more common approach for tic-tac-toe.

In [ ]:
(def board {   [0 0] nil,
               [0 1] nil,
               [0 2] nil,
               [1 0] nil,
               [1 1] nil,
               [1 2] nil,
               [2 0] nil,
               [2 1] nil,
               [2 2] nil
               })

## Helper functions
- It's possible to hardcode all of the winning sequences; however, this approach is not scalable in the case when this code has needs to be reused in a game simillar to tic-tac-toe but which has a bigger board.

In [ ]:
(defn get-all-coords []
    "Returns all of the coordinates for the board"
    (for [x (range 0 3) y (range 0 3)] 
      [x y]))

(defn get-rows-and-columns
    "It returns triplets of winning coordinates"
    []
    (let [ x (for [row [0 1 2]]
                 [(for [x [row] y [0 1 2]] [x y])])
           y (for [column [0 1 2]]
                 [(for [x [0 1 2] y [column]] [x y])])]
        (concat x y)))

(defn get-main-diag []
    "Sum on the main diagonal equals to 2"
    [(filter (fn [[x y]] (= 2 (+ x y))) (get-all-coords))])

(defn get-secondary-diag []
    "Different on the secondary diagonal equals to 0"
    [(filter (fn [[x y]] (= 0 (- x y))) (get-all-coords))])

## Functions for changing board state
- `is-valid`: is used to make sure that idx is within the allowable range.
- `make-move`

In [137]:
(defn is-valid [n]
    "Checks if n is withing the bounds"
    (and (>= n 0) (< n 3)))

(defn make-move
    "Puts a value into a board"
    [board i j e]
    (if (and (is-valid i) (is-valid j) (not (board [i j])))
        (assoc board [i j] e)
        (throw (Exception. "Invalid Move"))))

(def brd-x-win (make-move (make-move (make-move board 0 0 "x") 0 1 "x") 0 2 "x"))

(get-game-status brd-x-win)

:x-win

## Functions for evaluating board
- `get-count` : it's a helper function to get score for sequence of cells
- `get-game-status`

In [ ]:
(defn get-count
    [brd ids]
    (let [x (apply + (map (fn [id] (if (= "x" (brd id)) 1 0)) ids))
          o (apply + (map (fn [id] (if (= "o" (brd id)) 1 0)) ids))]
        {:x x :o o}))

(defn get-game-status
    "Returns one of [:x-winner|:o-winner|:draw|:in-progress]"
    [board]
    (let [coords (conj (get-rows-and-columns) (get-main-diag) (get-secondary-diag)) 
          stats  (map (fn [[ids]] (get-count board ids)) coords)
          x-win  (some #(= 3 (:x %)) stats)
          o-win  (some #(= 3 (:o %)) stats)]
          (cond
              (and x-win (not o-win)) :x-win
              (and (not x-win) o-win) :o-win
              (and x-win o-win) :draw
              :else :in-progress)))